# Image Filtering

# 图像滤波

## Overview

### Learning Objectives

* Becoming familiar with the Pythonic and Object-oriented ITK interfaces
* Understand the image processing pipeline model used in ITK
* Gain familiarity with some of the filtering algorithms available in ITK

### 学习目标

* 更加熟悉Pythonic和面向对象的ITK接口
* 理解使用ITK的图像处理pipeline模型
* 熟悉ITK中的一些滤波算法

## Data and Process Objects

## 数据和处理对象

While **data objects** (e.g., [Images](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch4.html#x40-440004.1) and [Meshes](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch4.html#x40-590004.3)) are used to represent data, **process objects** are classes that operate on data objects and may produce new data objects.

然而，**数据对象**，（如[Images](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch4.html#x40-440004.1) 和 [Meshes](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch4.html#x40-590004.3)) ，被用来标识数据，**处理对象**是用来处理数据对象的类，可能可以生成新的数据对象。

Process objects are classed as sources, filter objects, or mappers.

**Sources** (such as readers) produce data, **filter** objects take in data and process it to produce new data, and **mappers** accept data for output either to a file or some other system. 

Sometimes the term *filter* is used broadly to refer to all three types.

处理对象可以被分为sources, filter objects, 或者 mappers.

**Sources** （如readers）用来产生数据，**filter**对象输入数据，进行处理，生成新的数据，**mappers**接收用来接收输出到文件或其他系统的数据。

有些时候*filter*用来指代上面的所有三种类型。


![Data pipeline](data/data-pipeline.png)

Typically **data objects** and **process objects** are connected together using the `SetInput()` and `GetOutput()` methods.

Generation of new outputs and pixel data does not occur until the `Update()` method is called on the **end** of the pipeline (on the process object or the data object).

通常**data objects**和**process objects**通过`SetInput()`和`GetOutput()`方法连接到一块儿。

只有当流水线的终端（处理对象或数据对象）调用`Update()`方法之后，才会产生新的输出和像素数据。

![Pipeline updates](data/pipeline-updates.png)

The **data** associated with multi-dimensional images **is large and becoming larger**. 

Any practical image analysis system must address this fact in order to be useful in applications with multi-dimensional images. ITK addresses this problem via its **data streaming** facility.

多维图像关联的**数据**很大，并且正在变大。

任何实用的图像分析系统都必须解决这一问题，以便在多维图像的应用中发挥作用。ITK通过其**数据流**功能解决了这个问题。

![Streaming](data/streaming.gif)

Streaming is performed by splitting the image into non-overlapping **regions** at the **end of the pipeline**. The *RequestedRegion* then propagates up the pipeline.

在pipeline的结尾通过将图像分割成没有重叠的区域来执行数据流。*RequestedRegion* 会随着数据流向上传播。

There are three named [ImageRegion](https://itk.org/Insight/Doxygen/html/classitk_1_1ImageRegion.html)'s encountered in ITK:

- *BufferedRegion*: The region of pixels stored in memory

- *LargestPossibleRegion*: The largest possible region of an image

- *RequestedRegion*: The region requested on a single processing pass when streaming. The BufferedRegion and LargestPossibleRegion are as large or larger than the RequestedRegion

在ITK中会遇到三种[ImageRegion](https://itk.org/Insight/Doxygen/html/classitk_1_1ImageRegion.html):

- *BufferedRegion*. 像素的区域存储在内存中

- *LargestPossibleRegion：*一张图像最大的可能区域

- *RequestedRegion：*流式处理时在单个处理过程中请求的区域。BufferedRegion和LargestPossibleRegion与RequestedRegion一样大或更大。

For more information, see,

- The [*Filtering* chapter](https://itk.org/ITKSoftwareGuide/html/Book2/ITKSoftwareGuide-Book2ch2.html#x17-320002) of Book 2 of the ITK Software Guide.
- The [*Data Processing Pipeline* section](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch3.html#x39-420003.5) of the *System Overview* chapter of Book 1 of the ITK Software Guide.
- The [*Data Representation* chapter](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch4.html#x45-490004) of Book 1 of the ITK Software Guide.

## Tutorial

In [1]:
import numpy as np
import itk
from itkwidgets import view
from ipywidgets import interactive
import ipywidgets as widgets

We can monitor when a filter gets called by registering a command that gets called when [ProgressEvents](https://itk.org/Doxygen/html/classitk_1_1ProgressEvent.html) occur.

当[ProgressEvents](https://itk.org/Doxygen/html/classitk_1_1ProgressEvent.html)发生的时候，我们可以监测到filter被注册的命令调用。

In [2]:
itk.auto_progress(2)

Start an image processing pipeline with a source, an `ImageFileReader`.

以source（`ImageFileReader`）开始图像处理流水线。

In [3]:
file_name = 'data/PacMan.png'
reader = itk.ImageFileReader.New(FileName=file_name)

Loading ITKPyBase... done
Loading ITKCommon... done
Loading ITKIOImageBase... done
Loading ITKPyUtils... done


Next, let's create a smoothing filter. To connect the pipeline, specify the `Output` of the reader as the `Input` to the smoother.

接着，让我们创建一个平滑滤波器。为了连接pipeline，将reader的`Output`作为smoother的`Input`。

In [4]:
smoother = itk.RecursiveGaussianImageFilter.New(Input=reader.GetOutput())

Loading ITKStatistics... done
Loading ITKImageFilterBase... done
Loading ITKTransform... done
Loading ITKImageFunction... done
Loading ITKSmoothing... done


At this point, no output images have been generated.

We have configured the simple pipeline:

reader -> smoother

这个时候，没有生成任何的输出图像。

我们已经配置了简单的流水线：

reader -> smoother

In [5]:
print("reader's Output: %s" % reader.GetOutput())
print("smoother's Output: %s" % smoother.GetOutput())

reader's Output: Image (0x55a5bf3ea2b0)
  RTTI typeinfo:   itk::Image<unsigned char, 2u>
  Reference Count: 3
  Modified Time: 213
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0x55a5bf9052c0) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 0
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [0, 0]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [0, 0]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [0, 0]
  Spacing: [1, 1]
  Origin: [0, 0]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
1 0
0 1

  PointToIndexMatrix: 
1 0
0 1

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (0x55a5c0b11970)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, unsigned char>
      Reference Count: 1
      Modified Time: 210
      Debug: Off
      Object Name: 
      Observers: 
      

To generate the filter outputs, we must call `Update()` on the filter at the end of the pipeline. In this case, it is the smoother.

为了产生滤波的结果，我们必须调用filter的`Update()`方法。此处是smoother。

In [6]:
smoother.Update()

print("reader's Output: %s" % reader.GetOutput())
print("smoother's Output: %s" % smoother.GetOutput())

reader's Output: Image (0x55a5bf3ea2b0)
  RTTI typeinfo:   itk::Image<unsigned char, 2u>
  Reference Count: 3
  Modified Time: 394
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0x55a5bf9052c0) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 223
  UpdateMTime: 395
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  Spacing: [1, 1]
  Origin: [0, 0]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
1 0
0 1

  PointToIndexMatrix: 
1 0
0 1

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (0x55a5c0b11970)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, unsigned char>
      Reference Count: 1
      Modified Time: 392
      Debug: Off
      Object Name: 
      Ob

Running itkImageFileReaderIUC2... done
Running itkRecursiveGaussianImageFilterIUC2IUC2... done
done


Let's view the images.

让我们来看一下结果。

In [7]:
image = reader.GetOutput()
view(image, ui_collapsed=True)

Loading ITKBridgeNumPy... done


Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC2; proxy …

In [8]:
smoothed = smoother.GetOutput()
view(smoothed, ui_collapsed=True)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC2; proxy …

If we call `Update()` on the pipeline, the output pixel data is not needlessly generated because the pipeline is up-to-date.

如果我们调用流水线的`Update()`方法，由于管线是最新的，因此没有必要再生成输出pixel。

In [9]:
smoother.Update()

Running itkRecursiveGaussianImageFilterIUC2IUC2... done
done


However, if we change the amount of smoothing, new pixel data does need to be generated from the output of the smoother.

但是，如果我们改变了参数，那么就会再次起作用生成新的输出。

In [10]:
smoother.SetSigma(10.0)
smoother.Update()

Running itkRecursiveGaussianImageFilterIUC2IUC2... done
done


In [11]:
view(smoothed, ui_collapsed=True)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC2; proxy …

Note, however, that the reader does generate its output because it is up-to-date and upstream from the smoother.

If we artificially modify the reader, both the reader and the smoother need to regenerate their outputs.

但是，需要注意的是，reader并没有再次生成output，因为它是最新的，并不需要再次生成。

如果人为的改变reader，reader和smoother都需要重新生成输出。

In [12]:
reader.Modified()

smoother.Update()

Running itkImageFileReaderIUC2... done
Running itkRecursiveGaussianImageFilterIUC2IUC2... done
done


### Exercise 1: The effect of Sigma

Change the value of *Sigma* on the smoothing filter.

- How is the output effected?
- What are the units of Sigma?

相关链接，参考：https://itk.org/Doxygen/html/classitk_1_1RecursiveGaussianImageFilter.html

In [14]:
itk.RecursiveGaussianImageFilter?

In [19]:
fileName = 'data/PacMan.png'
reader = itk.ImageFileReader.New(FileName=fileName)
smoother = itk.RecursiveGaussianImageFilter.New(Input=reader.GetOutput())
smoother.SetSigma(6)
smoother.Update()
view(smoother.GetOutput())

Running itkImageFileReaderIUC2... done
Running itkRecursiveGaussianImageFilterIUC2IUC2... done
done


Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC2; proxy …

In [31]:
# %load solutions/3_Image_Filtering_Exercise1.py

### Exercise 2: Does setting a filter parameter to its current value cause regeneration of its output?

Call `smoother.SetSigma(smoother.GetSigma())` then `smoother.Update()`. 

- Is the output regenerated? 
- Is this expected / desirable?

**如何判断数据有没有被更新呢？看了一下通过时间貌似都不行**

In [22]:
smoother.Modified?

In [26]:
smoother.SetSigma(10)
smoother.Update()
print(smoother.GetOutput())
print("===========================")
smoother.Update()
print(smoother.GetOutput())
print("===========================")

Image (0x55a5c1170930)
  RTTI typeinfo:   itk::Image<unsigned char, 2u>
  Reference Count: 2
  Modified Time: 322
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0x55a5c0cf6440) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 2335
  UpdateMTime: 323
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  Spacing: [1, 1]
  Origin: [0, 0]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
1 0
0 1

  PointToIndexMatrix: 
1 0
0 1

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (0x55a5c0598a50)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, unsigned char>
      Reference Count: 4
      Modified Time: 321
      Debug: Off
      Object Name: 
      Observers: 
      

Running itkRecursiveGaussianImageFilterIUC2IUC2... done
done
Running itkRecursiveGaussianImageFilterIUC2IUC2... done
done


In [36]:
# %load solutions/3_Image_Filtering_Exercise2.py

### Exercise 3: Find other image filtering algorithms

The classes in ITK are organized into **Modules**, and collections of Modules are organized into **Groups**. Examine the [Image Smoothing](https://itk.org/Doxygen/html/group__ITKSmoothing.html) Module and the [Filtering](https://itk.org/Doxygen/html/group__Group-Filtering.html) Group. Can any other smoothing or denoising classes be found? 

### 练习3：找到其他图像滤波算法

ITK中的类被组织称**Modules**的形式，Modules的集合，组织称**Groups**。可以从[Image Smoothing](https://itk.org/Doxygen/html/group__ITKSmoothing.html)和[Filtering](https://itk.org/Doxygen/html/group__Group-Filtering.html)查找。使用一下其他的smoothing和denoising类。

In [27]:
file_name = 'data/PacMan.png'
reader = itk.ImageFileReader.New(FileName=file_name)

# `view` returns the widget object
viewer = view(image, annotations=False)

# Create an itk smoother filter object. By re-using the object, output memory-reallocation is avoided
smoother = itk.MedianImageFilter.New(reader.GetOutput())

# Define a function to use with ipywidgets `interactive`
def smooth_and_view(radius=2):
    smoother.SetRadius(radius)
    smoother.Update()
    # Update the image used in the viewer
    viewer.image = smoother.GetOutput()
slider = interactive(smooth_and_view, radius=(0, 10, 1))

widgets.VBox([viewer, slider])

In [40]:
# %load solutions/3_Image_Filtering_Exercise3.py

The answer example demonstrates how to combine `ipywidgets` native [interactive](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html) function to quickly create widgets coupled with an [`itkwidgets`](https://github.com/InsightSoftwareConsortium/itkwidgets) viewer. This is an effective method to explore algorithm parameters.

答案示例演示了如何使用`ipywidgets`。[interactive](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html)函数能够快速的创建[`itkwidgets`]相结合的widgets。这是一种探索算法参数的有效方法。

### Enjoy ITK!